# Import data

In [1]:
import pandas as pd

In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
data = pd.read_csv("C:/Users/kyungmyung/Desktop/panel_example.csv")

In [4]:
data.head(5)

,gvkey,year,ticker,cn,age,currentasset,totalasset,commonequity,cogs,commonoutstanding,...,totalliabilities,netincome,sales,stockholdersequity,advertisement_expense,operatingexpense,researchanddevelopment,marketvalue,pid,ptime
0,126554,1998,A,AGILENT TECHNOLOGIES INC,0,3075.0,4987.0,3022.0,3919.0,0.00000,...,1965.0,257.0,7952.0,3022.0,94.0,6917.0,948.0,0.000,1,11
1,126554,1999,A,AGILENT TECHNOLOGIES INC,1,3538.0,5444.0,3382.0,3862.0,380.00000,...,2062.0,512.0,8331.0,3382.0,130.0,7064.0,997.0,0.000,1,12
2,126554,2000,A,AGILENT TECHNOLOGIES INC,2,5655.0,8425.0,5265.0,5012.0,453.97601,...,3160.0,757.0,10773.0,5265.0,188.0,9204.0,1258.0,21024.764,1,13
3,126554,2001,A,AGILENT TECHNOLOGIES INC,3,4799.0,7986.0,5659.0,4353.0,461.00000,...,2327.0,174.0,8396.0,5659.0,115.0,8212.0,1349.0,10266.470,1,14
4,126554,2002,A,AGILENT TECHNOLOGIES INC,4,4880.0,8203.0,4627.0,2749.0,467.00000,...,3576.0,-1032.0,6010.0,4627.0,81.0,6408.0,1169.0,6421.250,1,15


In [5]:
# Define Dependent & Independent Variables

In [6]:
data.columns

Index(['gvkey', 'year', 'ticker', 'cn', 'age', 'currentasset', 'totalasset',
       'commonequity', 'cogs', 'commonoutstanding', 'longtermdebt', 'employ',
       'inventories', 'currentliabilities', 'liabilitiesequity',
       'totalliabilities', 'netincome', 'sales', 'stockholdersequity',
       'advertisement_expense', 'operatingexpense', 'researchanddevelopment',
       'marketvalue', 'pid', 'ptime'],
      dtype='object')

In [8]:
analdata = data[["marketvalue","cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset','gvkey','year']]

# Create Year and Firm Dummy Variables

In [10]:
year = pd.Categorical(data['year'])
firm = pd.Categorical(data['gvkey'])

In [11]:
# Add dummy to the analdata

In [12]:
analdata = analdata.set_index(['gvkey','year'])

In [13]:
analdata.head(5)

marketvalue    cogs     employ  netincome  \
gvkey  year                                              
126554 1998        0.000  3919.0   0.000000      257.0   
       1999        0.000  3862.0  36.400002      512.0   
       2000    21024.764  5012.0  47.000000      757.0   
       2001    10266.470  4353.0  41.000000      174.0   
       2002     6421.250  2749.0  36.000000    -1032.0   

             researchanddevelopment  advertisement_expense  totalasset  
gvkey  year                                                             
126554 1998                   948.0                   94.0      4987.0  
       1999                   997.0                  130.0      5444.0  
       2000                  1258.0                  188.0      8425.0  
       2001                  1349.0                  115.0      7986.0  
       2002                  1169.0                   81.0      8203.0

In [14]:
dv = analdata["marketvalue"]

In [15]:
iv = analdata[["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']]

# Correlation Check

In [17]:
analdata.corr(method='pearson')

,marketvalue,cogs,employ,netincome,researchanddevelopment,advertisement_expense,totalasset
marketvalue,1.000000,0.443326,0.400503,0.685736,0.622753,0.467460,0.392121
cogs,0.443326,1.000000,0.853080,0.364853,0.374472,0.569667,0.291798
employ,0.400503,0.853080,1.000000,0.329034,0.247207,0.447012,0.248023
netincome,0.685736,0.364853,0.329034,1.000000,0.395915,0.367995,0.444155
researchanddevelopment,0.622753,0.374472,0.247207,0.395915,1.000000,0.572667,0.084360
advertisement_expense,0.467460,0.569667,0.447012,0.367995,0.572667,1.000000,0.316115
totalasset,0.392121,0.291798,0.248023,0.444155,0.084360,0.316115,1.000000


# Multicollinearity Check Function

In [19]:
def Coll(data):
    Collinearity = pd.DataFrame()
    Collinearity['indvariables']= data.columns
    Collinearity['VIF']= [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return Collinearity

In [20]:
Coll(iv)

,indvariables,VIF
0,cogs,5.071093
1,employ,4.199243
2,netincome,1.643974
3,researchanddevelopment,1.839094
4,advertisement_expense,2.177635
5,totalasset,1.434863


# FE and RE Model

In [40]:
# !pip install linearmodels

In [41]:
# !pip install statsmodels

In [24]:
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
import statsmodels.api as sm

In [25]:
# FE Model

In [28]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])

In [29]:
entitytimefixedmodel = PanelOLS(analdata.marketvalue, iv, entity_effects=True, drop_absorbed=True, time_effects=True)
fe_res = entitytimefixedmodel.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            marketvalue   R-squared:                        0.5899
Estimator:                   PanelOLS   R-squared (Between):              0.5403
No. Observations:                6637   R-squared (Within):               0.6306
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.5819
Time:                        20:01:43   Log-likelihood                -7.696e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1508.9
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6294)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             1508.9
                            

In [30]:
# RE Model

In [31]:
random_model = RandomEffects(analdata.marketvalue, iv)
re_res = random_model.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:            marketvalue   R-squared:                        0.6227
Estimator:              RandomEffects   R-squared (Between):              0.5963
No. Observations:                6637   R-squared (Within):               0.6272
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.6082
Time:                        20:01:48   Log-likelihood                -7.755e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1824.1
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6630)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             1834.2
                            

# Hausman Test

In [32]:
import numpy.linalg as la
from scipy import stats
import numpy as np

In [33]:
def hausman(fe, re):
    b = fe.params
    B = re.params
    v_b = fe.cov
    v_B = re.cov
    df = b[np.abs(b) < 1e8].size
    chi2 = np.dot((b - B).T, la.inv(v_b - v_B).dot(b - B)) 
    pval = stats.chi2.sf(chi2, df)
    return chi2, df, pval

In [34]:
hausman_results = hausman(fe_res, re_res) 
print("chi-Squared:" + str(hausman_results[0]))
print('degrees of freedom:' + str(hausman_results[1]))
print('p-Value: ' + str(hausman_results[2]))

# P-value is very small 1.0537386319928037e-17 is approximately 0.00000004362.
# This means that the null hypothesis can be rejcted. 
# The Null Hypothesis = There are no correlation between unique errors and the regressors
# The FE-model is a more efficient estimator than the RE-model because we have endogeneity in our model. 

chi-Squared:95.20102522415266
degrees of freedom:7
p-Value: 1.0537386319928037e-17


# Breusch-Godfrey Test

In [50]:
# The Null Hypothesis = There is no correlation among the residuals.

In [51]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])

In [54]:
OLS = sm.OLS(analdata.marketvalue, iv).fit()
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     2065.
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        20:23:30   Log-Likelihood:                -78850.
No. Observations:                6637   AIC:                         1.577e+05
Df Residuals:                    6630   BIC:                         1.578e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1187

In [57]:
import statsmodels.stats.diagnostic as dg
#perform Breusch-Godfrey test at order p = 5
print(dg.acorr_breusch_godfrey(OLS, nlags=10))

(2951.3986604969905, 0.0, 530.1240512118089, 0.0)


In [58]:
# Test statistic = 2951.3986604969905
# P-value = 0.0
# Conclusion: we cannot reject the null hypothesis. There is no serial correlation among residuals

# Robust Standard Errors

In [60]:
# heteroskedasticity corrected standard errors

In [ ]:
# Eicker-Huber-White robust standard errors, aka “HC2” 

In [61]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
OLS = sm.OLS(analdata.marketvalue, iv).fit(cov_type="HC2")
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     171.8
Date:                Mon, 19 Sep 2022   Prob (F-statistic):          8.83e-204
Time:                        20:30:27   Log-Likelihood:                -78850.
No. Observations:                6637   AIC:                         1.577e+05
Df Residuals:                    6630   BIC:                         1.578e+05
Df Model:                           6                                         
Covariance Type:                  HC2                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1187

In [ ]:
# Not used the robust

In [69]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
OLS = sm.OLS(analdata.marketvalue, iv).fit()
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     2065.
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        20:34:04   Log-Likelihood:                -78850.
No. Observations:                6637   AIC:                         1.577e+05
Df Residuals:                    6630   BIC:                         1.578e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1187

In [ ]:
# The FE model

In [64]:
# “robust” - Control for heteroskedasticity using White’s estimator

In [65]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
entitytimefixedmodel = PanelOLS(analdata.marketvalue, iv, entity_effects=True, drop_absorbed=True, time_effects=True)
fe_res = entitytimefixedmodel.fit(cov_type="robust")
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            marketvalue   R-squared:                        0.5899
Estimator:                   PanelOLS   R-squared (Between):              0.5403
No. Observations:                6637   R-squared (Within):               0.6306
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.5819
Time:                        20:32:44   Log-likelihood                -7.696e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      1508.9
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6294)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             259.91
                            

In [67]:
# Not used the "robust"

In [68]:
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
entitytimefixedmodel = PanelOLS(analdata.marketvalue, iv, entity_effects=True, drop_absorbed=True, time_effects=True)
fe_res = entitytimefixedmodel.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            marketvalue   R-squared:                        0.5899
Estimator:                   PanelOLS   R-squared (Between):              0.5403
No. Observations:                6637   R-squared (Within):               0.6306
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.5819
Time:                        20:33:33   Log-likelihood                -7.696e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1508.9
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6294)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             1508.9
                            

In [73]:
# Clustered standard errors Often, we know something about the structure of likely errors, namely that they occur in groups. 

In [71]:
# Used firm cluster errors
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
OLS = sm.OLS(analdata.marketvalue, iv).fit(cov_type="cluster", cov_kwds={"groups":firm})
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     35.94
Date:                Mon, 19 Sep 2022   Prob (F-statistic):           7.97e-33
Time:                        20:36:04   Log-Likelihood:                -78850.
No. Observations:                6637   AIC:                         1.577e+05
Df Residuals:                    6630   BIC:                         1.578e+05
Df Model:                           6                                         
Covariance Type:              cluster                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1187

In [72]:
# Not used the cluster error
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
OLS = sm.OLS(analdata.marketvalue, iv).fit()
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            marketvalue   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     2065.
Date:                Mon, 19 Sep 2022   Prob (F-statistic):               0.00
Time:                        20:36:20   Log-Likelihood:                -78850.
No. Observations:                6637   AIC:                         1.577e+05
Df Residuals:                    6630   BIC:                         1.578e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                   1187

In [74]:
# Used firm cluster errors for the FE-model

In [76]:
# Firm cluster error
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
entitytimefixedmodel = PanelOLS(analdata.marketvalue, iv, entity_effects=True, drop_absorbed=True, time_effects=True)
fe_res = entitytimefixedmodel.fit(cov_type="clustered", cluster_entity=True)
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            marketvalue   R-squared:                        0.5899
Estimator:                   PanelOLS   R-squared (Between):              0.5403
No. Observations:                6637   R-squared (Within):               0.6306
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.5819
Time:                        20:37:49   Log-likelihood                -7.696e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1508.9
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6294)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             71.425
                            

In [77]:
# Not used the cluster error
iv_vars = ["cogs",'employ','netincome','researchanddevelopment','advertisement_expense','totalasset']
iv = sm.add_constant(analdata[iv_vars])
entitytimefixedmodel = PanelOLS(analdata.marketvalue, iv, entity_effects=True, drop_absorbed=True, time_effects=True)
fe_res = entitytimefixedmodel.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:            marketvalue   R-squared:                        0.5899
Estimator:                   PanelOLS   R-squared (Between):              0.5403
No. Observations:                6637   R-squared (Within):               0.6306
Date:                Mon, Sep 19 2022   R-squared (Overall):              0.5819
Time:                        20:38:03   Log-likelihood                -7.696e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1508.9
Entities:                         306   P-value                           0.0000
Avg Obs:                       21.690   Distribution:                  F(6,6294)
Min Obs:                       2.0000                                           
Max Obs:                       32.000   F-statistic (robust):             1508.9
                            